In [1]:
import numpy as np
import sys
import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

/Users/yuhaofei/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
print(os.listdir('./kaggle_input'))

['application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


## Feature Engineering 1
application dataset
polynomial feature engineering + domain knowledge  feature engineering

In [3]:
app_train = pd.read_csv('./kaggle_input/application_train.csv')
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
app_test = pd.read_csv('./kaggle_input/application_test.csv')
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
label_count = 0

for col in app_train:
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            label.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = label.transform(app_train[col])
            app_test[col] = label.transform(app_test[col])
            label_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)
print(app_train.shape)
print(app_test.shape)

(307511, 243)
(48744, 239)


In [6]:
train_labels = app_train['TARGET']
app_train,app_test = app_train.align(app_test, join = 'inner' ,  axis = 1)
app_train['TARGET'] = train_labels
print(app_train.shape)
print(app_test.shape)

(307511, 240)
(48744, 239)


In [7]:
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import PolynomialFeatures  
# do poly feature engineering using combination of 5 features
poly_features_train = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

poly_target = poly_features_train['TARGET']
poly_features_train = poly_features_train.drop(columns = ['TARGET'])

# fill NaN in the table
imputer = Imputer(strategy = 'median')
poly_features_train = imputer.fit_transform(poly_features_train)
poly_features_test = imputer.transform(poly_features_test)
                        
poly_transformer = PolynomialFeatures(degree = 4)

# Train the polynomial features
poly_transformer.fit(poly_features_train)

# Transform the features
poly_features_train = poly_transformer.transform(poly_features_train)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features_train.shape)

poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

Polynomial Features shape:  (307511, 70)


['1',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'DAYS_BIRTH',
 'EXT_SOURCE_1^2',
 'EXT_SOURCE_1 EXT_SOURCE_2',
 'EXT_SOURCE_1 EXT_SOURCE_3',
 'EXT_SOURCE_1 DAYS_BIRTH',
 'EXT_SOURCE_2^2',
 'EXT_SOURCE_2 EXT_SOURCE_3',
 'EXT_SOURCE_2 DAYS_BIRTH',
 'EXT_SOURCE_3^2',
 'EXT_SOURCE_3 DAYS_BIRTH',
 'DAYS_BIRTH^2']

In [8]:
poly_features_train = pd.DataFrame(poly_features_train, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))


# Add in the target
poly_features_train['TARGET'] = poly_target


# Merge polynomial features into training dataframe
poly_features_train['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features_train, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

Training data with polynomial features shape:  (307511, 309)
Testing data with polynomial features shape:   (48744, 309)


In [9]:
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

# set domain knowledge feature for train and set
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

app_train = pd.merge(app_train_domain,poly_features_train, how = 'inner', on='SK_ID_CURR')
app_test  = pd.merge(app_test_domain,poly_features_test, how = 'inner', on='SK_ID_CURR')

# align
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

print('Training data with domain features shape: ', app_train.shape)
print('Testing data with domain features shape:  ', app_test.shape)

Training data with domain features shape:  (307511, 313)
Testing data with domain features shape:   (48744, 313)


## Feature Engineering 2
bureau dataset
aggregation feature engineering

In [10]:
bureau = pd.read_csv('./kaggle_input/bureau.csv')
bb = pd.read_csv('./kaggle_input/bureau_balance.csv')
print(bureau.shape)
print(bb.shape)

(1716428, 17)
(27299925, 3)


In [11]:
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()    

,SK_ID_CURR,previous_loan_counts
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [12]:
print(app_train.shape)
print(app_test.shape)
app_train = app_train.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

# Fill the missing values with 0 
app_train['previous_loan_counts'] = app_train['previous_loan_counts'].fillna(0)
app_test['previous_loan_counts'] = app_test['previous_loan_counts'].fillna(0)
print(app_train.shape)
print(app_test.shape)
app_train.head()

(307511, 313)
(48744, 313)
(307511, 314)
(48744, 314)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,EXT_SOURCE_2 EXT_SOURCE_3^3,EXT_SOURCE_2 EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH^2,EXT_SOURCE_2 DAYS_BIRTH^3,EXT_SOURCE_3^4,EXT_SOURCE_3^3 DAYS_BIRTH,EXT_SOURCE_3^2 DAYS_BIRTH^2,EXT_SOURCE_3 DAYS_BIRTH^3,DAYS_BIRTH^4,previous_loan_counts
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0.000712,-48.326185,3.280441e+06,-2.226804e+11,0.000377,-25.615272,1.738796e+06,-1.180316e+11,8.012133e+15,8.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0.095432,-2988.969365,9.361535e+07,-2.932058e+12,0.082094,-2571.209606,8.053099e+07,-2.522253e+12,7.899766e+16,4.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0.215874,-5635.596478,1.471224e+08,-3.840763e+12,0.283309,-7396.031846,1.930801e+08,-5.040532e+12,1.315876e+17,2.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.099757,-3541.866544,1.257541e+08,-4.464903e+12,0.082094,-2914.753269,1.034884e+08,-3.674359e+12,1.304582e+17,0.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0.049498,-1843.135727,6.863256e+07,-2.555660e+12,0.082094,-3056.925134,1.138303e+08,-4.238680e+12,1.578351e+17,1.0


In [13]:
# Group by the client id, calculate aggregation statistics
bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()

SK_ID_CURR DAYS_CREDIT                               CREDIT_DAY_OVERDUE  \
                   count         mean  max   min   sum              count   
0     100001           7  -735.000000  -49 -1572 -5145                  7   
1     100002           8  -874.000000 -103 -1437 -6992                  8   
2     100003           4 -1400.750000 -606 -2586 -5603                  4   
3     100004           2  -867.000000 -408 -1326 -1734                  2   
4     100005           3  -190.666667  -62  -373  -572                  3   

                ... DAYS_CREDIT_UPDATE                               \
  mean max min  ...              count        mean  max   min   sum   
0  0.0   0   0  ...                  7  -93.142857   -6  -155  -652   
1  0.0   0   0  ...                  8 -499.875000   -7 -1185 -3999   
2  0.0   0   0  ...                  4 -816.000000  -43 -2131 -3264   
3  0.0   0   0  ...                  2 -532.000000 -382  -682 -1064   
4  0.0   0   0  ...                  3  -54.333333  -11  -121  -163   

  AMT_ANNUITY                                      
        count         mean      max  min      sum  
0           7  3545.357143  10822.5  0.0  24817.5  
1           7     0.000000      0.0  0.0      0.0  
2           0          NaN      NaN  NaN      0.0  
3           0          NaN      NaN  NaN      0.0  
4           3  1420.500000   4261.5  0.0   4261.5  

[5 rows x 61 columns]

In [14]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_%s_%s' % (var, stat))
bureau_agg.columns = columns
bureau_agg.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


In [15]:
print(app_train.shape)
print(app_test.shape)
app_train = app_train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

print(app_train.shape)
print(app_test.shape)
app_train.head()

(307511, 314)
(48744, 314)
(307511, 374)
(48744, 374)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,8.0,-499.875,-7.0,-1185.0,-3999.0,7.0,0.0,0.0,0.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,4.0,-816.000,-43.0,-2131.0,-3264.0,0.0,NaN,NaN,NaN,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,2.0,-532.000,-382.0,-682.0,-1064.0,0.0,NaN,NaN,NaN,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,1.0,-783.000,-783.0,-783.0,-783.0,0.0,NaN,NaN,NaN,0.0


## Feature Engineering 3
POS_CASH_balance.csv feature engineering

In [16]:
pos = pd.read_csv('./kaggle_input/POS_CASH_balance.csv')
print(pos.shape)
pos.head()

(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [17]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
# Features
aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}
for cat in cat_cols:
    aggregations[cat] = ['mean']
pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
# Count pos cash accounts
pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
del pos
gc.collect()

7

In [18]:
app_train = app_train.merge(pos_agg, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(pos_agg, on = 'SK_ID_CURR', how = 'left')
app_train.shape,app_test.shape

((307511, 392), (48744, 392))

## Feature Engineering 4
installments_payments.csv

In [19]:
ins = pd.read_csv('./kaggle_input/installments_payments.csv')
print(ins.shape)

(13605401, 8)


In [20]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)

In [21]:
ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)

aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}
for cat in cat_cols:
    aggregations[cat] = ['mean']
ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
del ins
gc.collect()

0

In [22]:
app_train = app_train.merge(ins_agg, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(ins_agg, on = 'SK_ID_CURR', how = 'left')
app_train.shape,app_test.shape

((307511, 418), (48744, 418))

## Feature Engineering 5
credit_card_balance.csv

In [23]:
cc = pd.read_csv('./kaggle_input/credit_card_balance.csv')
print(cc.shape)

(3840312, 23)


In [24]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)

In [25]:
# General aggregations
cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
# Count credit card lines
cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
del cc
gc.collect()

0

In [26]:
app_train = app_train.merge(cc_agg, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(cc_agg, on = 'SK_ID_CURR', how = 'left')
app_train.shape,app_test.shape

((307511, 559), (48744, 559))

## Feature Engineering 6
previous_applications.csv

In [27]:
prev = pd.read_csv('./kaggle_input/previous_application.csv')
prev_col = prev.columns.tolist()
prev.shape

(1670214, 37)

In [28]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)

In [29]:
# fill NaN in the table
prev_col = []
for i in prev:
    prev_col.append(i)
imputer = Imputer(missing_values=np.nan,strategy = 'constant',fill_value = 32575)
prev = imputer.fit_transform(prev)
print(prev)

[[2.0304950e+06 2.7187700e+05 1.7304300e+03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.8024250e+06 1.0812900e+05 2.5188615e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.5234660e+06 1.2204000e+05 1.5060735e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [2.6596320e+06 2.4954400e+05 1.1520855e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.7855820e+06 4.0031700e+05 1.8821520e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.4187620e+06 2.6121200e+05 1.6431300e+04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]


In [30]:
prev = pd.DataFrame(prev,columns = prev_col)
prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

In [31]:
# Previous applications numeric features
num_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_APPLICATION': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
}
# Previous applications categorical features
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']

prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
# Previous Applications: Approved Applications - only numerical features
approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
# Previous Applications: Refused Applications - only numerical features
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
del refused, refused_agg, approved, approved_agg, prev
gc.collect()

0

In [32]:
app_train = app_train.merge(prev_agg, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(prev_agg, on = 'SK_ID_CURR', how = 'left')
app_test.replace(to_replace=np.NaN, value=10000, inplace=True)
app_train.replace(to_replace=np.NaN, value=10000, inplace=True)
app_test.replace(to_replace=np.inf, value=10000, inplace=True)
app_train.replace(to_replace=np.inf, value=10000, inplace=True)

In [33]:
print(app_train.shape)
print(app_test.shape)

(307511, 808)
(48744, 808)


## Prepare for Model Part

In [34]:
new_corrs = []
columns = app_train.columns.values.tolist()
# Iterate through the columns 
for col in columns:
    # Calculate correlation with the target
    corr = train_labels.corr(app_train[col])
    
    # Append the list as a tuple

    new_corrs.append((col, abs(corr)))
new_corrs = sorted(new_corrs, key = lambda x: (x[1]), reverse = False)
print(new_corrs[0][0])
new_corrs[:300]

ORGANIZATION_TYPE_Industry: type 7


[('ORGANIZATION_TYPE_Industry: type 7', 9.405271151196014e-05),
 ('ORGANIZATION_TYPE_Advertising', 0.00011748220212479868),
 ('FLAG_DOCUMENT_20', 0.00021539355797725417),
 ('ORGANIZATION_TYPE_Legal Services', 0.0002359890687348436),
 ('INSTAL_PAYMENT_PERC_VAR', 0.00029003204294170815),
 ('NAME_HOUSING_TYPE_Co-op apartment', 0.0003123669248053774),
 ('FLAG_DOCUMENT_5', 0.00031577741664628545),
 ('ORGANIZATION_TYPE_Business Entity Type 1', 0.0003386506512333853),
 ('FLAG_CONT_MOBILE', 0.000370126802348927),
 ('OCCUPATION_TYPE_Realty agents', 0.00039357160127685833),
 ('EXT_SOURCE_2_x', 0.0003965474753989494),
 ('CNT_FAM_MEMBERS', 0.00042303514898676855),
 ('WEEKDAY_APPR_PROCESS_START_THURSDAY', 0.00044588677360102026),
 ('NAME_TYPE_SUITE_Group of people', 0.0004515676330592704),
 ('ORGANIZATION_TYPE_Housing', 0.0004642395738526417),
 ('FLAG_MOBIL', 0.0005343955790282575),
 ('bureau_AMT_CREDIT_SUM_DEBT_min', 0.0005566982285586807),
 ('bureau_AMT_ANNUITY_mean', 0.0005985960863499923),
 ('W

In [35]:
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
import numpy as np
for i in range(0):
    if (new_corrs[i][0] == 'SK_ID_CURR'):
        continue
    app_train = app_train.drop(columns = [new_corrs[i][0]])
    app_test = app_test.drop(columns = [new_corrs[i][0]])

train = app_train.drop(columns = ['SK_ID_CURR'])
test  = app_test.drop(columns = ['SK_ID_CURR'])
#scaler = MinMaxScaler(feature_range = (0, 1))
scaler = Normalizer()
imputer = Imputer(strategy = 'median')

imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)
print(train)

[[ 0.00000000e+00  0.00000000e+00  1.24810702e-16 ...  1.24810702e-12
   1.24810702e-12  1.24810702e-12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.26586026e-13
   1.26586026e-13  1.26586026e-13]
 [ 7.59949754e-18  7.59949754e-18  7.59949754e-18 ...  7.59949754e-14
   7.59949754e-14  7.59949754e-14]
 ...
 [ 0.00000000e+00  0.00000000e+00  1.99332005e-17 ...  1.99332005e-13
   1.99332005e-13  1.99332005e-13]
 [ 0.00000000e+00  0.00000000e+00  4.88573644e-17 ...  4.88573644e-13
   4.88573644e-13  4.88573644e-13]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.88346552e-15
   3.71623494e-16  7.43246989e-16]]


In [36]:
train = np.array(train)
test  = np.array(test)
train_labels = np.array(train_labels)
print(train.shape)
print(test.shape)

(307511, 807)
(48744, 807)


## Model Part

In [37]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

def evaluate_AUC(y_pred,y_true):
    print(roc_auc_score(y_true,y_pred))

def get_train_test_dataset(total_train,total_train_labels):
    X_train, X_test, y_train, y_test = train_test_split(total_train, total_train_labels, test_size=0.3, random_state=31)
    return X_train, X_test, y_train, y_test

In [38]:
X_train, X_test, y_train, y_test = get_train_test_dataset(train,train_labels)

In [39]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1,)

clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.793209	training's binary_logloss: 0.23815	valid_1's auc: 0.762954	valid_1's binary_logloss: 0.242471
[400]	training's auc: 0.817387	training's binary_logloss: 0.228372	valid_1's auc: 0.771999	valid_1's binary_logloss: 0.239088
[600]	training's auc: 0.833298	training's binary_logloss: 0.221751	valid_1's auc: 0.775445	valid_1's binary_logloss: 0.237877
[800]	training's auc: 0.846537	training's binary_logloss: 0.216135	valid_1's auc: 0.777101	valid_1's binary_logloss: 0.237306
[1000]	training's auc: 0.858087	training's binary_logloss: 0.211123	valid_1's auc: 0.777795	valid_1's binary_logloss: 0.237055
[1200]	training's auc: 0.868002	training's binary_logloss: 0.206656	valid_1's auc: 0.778431	valid_1's binary_logloss: 0.236845
[1400]	training's auc: 0.877376	training's binary_logloss: 0.20234	valid_1's auc: 0.778577	valid_1's binary_logloss: 0.236799
[1600]	training's auc: 0.886194	training's binary_loglo

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.9497036, importance_type='split',
               learning_rate=0.02, max_depth=8, min_child_samples=20,
               min_child_weight=39.3259775, min_split_gain=0.0222415,
               n_estimators=10000, n_jobs=-1, nthread=4, num_leaves=34,
               objective=None, random_state=None, reg_alpha=0.041545473,
               reg_lambda=0.0735294, silent=-1, subsample=0.8715623,
               subsample_for_bin=200000, subsample_freq=0, verbose=-1)

In [40]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

model = XGBClassifier(learning_rate = 0.05, n_estimators=20, max_depth=5)
model.fit(X_train,y_train)
y_pred = model.predict_proba(X_test)[:, 1]
evaluate_AUC(y_pred,y_test)

model = GradientBoostingClassifier(learning_rate=0.3,n_estimators=10,max_depth=2)
model.fit(X_train,y_train)
y_pred = model.predict_proba(X_test)[:, 1]
evaluate_AUC(y_pred,y_test)

model = LogisticRegression(C = 0.0001)
model.fit(X_train,y_train)
y_pred = model.predict_proba(X_test)[:, 1]
evaluate_AUC(y_pred,y_test)


model.fit(X_train,y_train)
y_pred = model.predict_proba(X_test)[:, 1]
evaluate_AUC(y_pred,y_test)



0.7271734420139127
0.5024053004312774
0.7062332640896789
0.7062332640896789


In [41]:
final_predict = app_test_poly[['SK_ID_CURR']]
y_evaluate_label = clf.predict_proba(test)[:, 1]
final_predict["TARGET"] = y_evaluate_label
final_predict.head()

,SK_ID_CURR,TARGET
0,100001,0.064334
1,100005,0.154204
2,100013,0.037086
3,100028,0.024925
4,100038,0.137759


In [42]:
final_predict.to_csv("resultkaggle.csv",index=False)